<a href="https://colab.research.google.com/github/Konstantin5054232/ausbildungsprojekte/blob/main/13_toxische_kommentare/toxische_kommentare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Klassifizierung von Kommentaren in positiv und negativ

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества F1 не меньше 0.75.

## Datenaufbereitung

In [ ]:
# Wir importieren die notwendigen Bibliotheken.
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from pymystem3 import Mystem
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Wir werden die Daten herunterladen und untersuchen.
data = pd.read_csv('/content/toxic_comments.csv')
display(data.shape)
display(data.head())
data.info()

(159292, 3)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [ ]:
# Wir werden das Klassengleichgewicht studieren.
balance = data['toxic'].sum() / len(data)
balance

0.10161213369158527

Wir sehen, dass die Klassen unausgewogen sind: etwa 10 % toxische Kommentare.

In [ ]:
# Wir werden Funktionen für die Lemmatisierung von Text und das Entfernen aller Zeichen außer englischen Buchstaben schreiben.
corpus = data['text']

def lemmatize(text):
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)
    return lemm_text


def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split())

data['lemm_text'] = corpus.apply(clear_text)

In [ ]:
# Wir teilen die Tabelle in Trainings- und Testbeispiele auf.
data_train, data_test = train_test_split(data, test_size=0.25, random_state=12345)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Wir berechnen die TF-IDF für das Trainings- und Testkorpus.
train_corpus = data_train['lemm_text'].values.astype('U')
train_labels = data_train['toxic']
 
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
 
tfidf_train = count_tf_idf.fit_transform(train_corpus)


test_corpus = data_test['lemm_text'].values.astype('U')
test_labels = data_test['toxic']

tfidf_test = count_tf_idf.transform(test_corpus)

## Modelltraining

Wir werden 3 verschiedene Modelle trainieren, Prognosen erstellen und die f1-Metrik für die Testprobe berechnen.

In [ ]:
model_1 = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model_1.fit(tfidf_train, train_labels)
predictions = model_1.predict(tfidf_test)
 
print(f1_score(test_labels, predictions))

0.7529845886694161


In [ ]:
model_2 = DecisionTreeClassifier()
model_2.fit(tfidf_train, train_labels)
predictions = model_2.predict(tfidf_test)
 
print(f1_score(test_labels, predictions))

0.7191404297851075


In [10]:
model_3 = RandomForestClassifier()
model_3.fit(tfidf_train, train_labels)
predictions = model_3.predict(tfidf_test)
 
print(f1_score(test_labels, predictions))

0.714503351614869


## Schlussfolgerungen

Wir haben drei verschiedene LogisticRegression-, DecisionTreeClassifier- und RandomForestClassifier-Modelle trainiert. Für jedes Modell wurden Prognosen erstellt und die Metrik f1 berechnet. Der größte Wert der f1-Metrik wurde durch das LogisticRegression-Modell erhalten, der kleinste durch das RandomForestClassifier-Modell.

Mit dem geschriebenen Code können wir Kommentare in positive und negative klassifizieren, ohne Zeit mit dem Lesen zu verschwenden, was den Mitarbeitern des Online-Shops Arbeitszeit bei der Bewertung von Kommentaren spart.